In [1]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numba
from typing import Tuple, List
from matchms import Spectrum
from matchms.typing import SpectrumType
import numpy as np
import pandas as pd
from pathlib import Path
import json
from numba import cuda
from matchms import Spectrum
from numba.cuda.cudadrv.devicearray import DeviceNDArray
from numba import types
import math

np.set_printoptions(precision=3)

from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses

def process_spectrum(spectrum):
    spectrum = select_by_mz(spectrum, mz_from=10.0, mz_to=1000.0)
    spectrum = normalize_intensities(spectrum)
    spectrum = select_by_relative_intensity(spectrum, intensity_from=0.001)
    spectrum = reduce_to_number_of_peaks(spectrum, n_max=1000)
    spectrum = require_minimum_number_of_peaks(spectrum, n_required=5)
    return spectrum


def get_ref_spectra_from_df(spectra_df, limit=None):
    # This function will take a dataframe with spectra and return a list of matchms spectra
    # Argh, This function is annoyingly slow. Added simple parallelization.
    
    # for index, row in spectra_df.iterrows():
    def fn(index, row):
        pbid = row["pbid"]
        precursor_mz = row["precursor_mz"]
        smiles = row["pb_smiles"]
        inchikey = row["pb_inchikey"]
        mz_array = np.array(json.loads(row["peaks_mz"]))
        intensity_array = np.array(json.loads(row["peaks_intensities"]))
        sp = Spectrum(mz=mz_array, intensities=intensity_array,
                        metadata={'id': pbid, 
                                'precursor_mz': precursor_mz, 
                                'smiles': smiles, 
                                'inchikey': inchikey}) 
        sp = process_spectrum(sp)
        return sp
    if limit is not None:
        spectra_df = spectra_df.head(limit)
    spectra = Parallel(-2)(delayed(fn)(index, row) for index, row in tqdm(spectra_df.iterrows(), total=len(spectra_df)) )
    spectra = [s for s in spectra if s is not None]
    return spectra

def spectra_peaks_to_tensor(spectra: list, fill: float):
    sp_max_shape = max(len(s.peaks) for s in spectra)
    mz = np.full((len(spectra), sp_max_shape), fill, 'float32')
    int = np.full((len(spectra), sp_max_shape), fill, 'float32')
    batch = np.zeros(len(spectra),dtype=np.int32)
    for i, s in enumerate(spectra):
        arr = s.peaks.to_numpy
        mz[i, :len(s.peaks)] = arr[...,0] 
        int[i, :len(s.peaks)] = arr[...,1]
        batch[i] = len(s.peaks)
    spec = np.stack([mz, int], axis=0)
    return spec, batch

In [2]:
ref_spectra_df_path = Path("data/input/example_dataset_tornike.csv")
ref_spectra_df = pd.read_csv(ref_spectra_df_path)
large_references = get_ref_spectra_from_df(ref_spectra_df, limit=2048 * 2)

100%|██████████| 4096/4096 [00:03<00:00, 1264.23it/s]


In [22]:
import time

R = 1024 
Q = 1024

references = large_references[Q:Q+R]
queries = large_references[:Q]

print(f"Total iterations: {len(queries) * len(references)}")

Total iterations: 1048576


## Regular CPU-bound run:

In [42]:
import time
from itertools import product

@numba.njit
def find_matches(spec1_mz: np.ndarray, spec2_mz: np.ndarray,
                 tolerance: float, shift: float = 0) -> List[Tuple[int, int]]:
    """Faster search for matching peaks.
    Makes use of the fact that spec1 and spec2 contain ordered peak m/z (from
    low to high m/z).

    Parameters
    ----------
    spec1_mz:
        Spectrum peak m/z values as numpy array. Peak mz values must be ordered.
    spec2_mz:
        Spectrum peak m/z values as numpy array. Peak mz values must be ordered.
    tolerance
        Peaks will be considered a match when <= tolerance appart.
    shift
        Shift peaks of second spectra by shift. The default is 0.

    Returns
    -------
    matches
        List containing entries of type (idx1, idx2).

    """
    
    lowest_idx = 0
    matches = []
    for peak1_idx in range(spec1_mz.shape[0]):
        mz = spec1_mz[peak1_idx]
        low_bound = mz - tolerance
        high_bound = mz + tolerance
        for peak2_idx in range(lowest_idx, spec2_mz.shape[0]):
            mz2 = spec2_mz[peak2_idx] + shift
            if mz2 > high_bound:
                break
            if mz2 < low_bound:
                lowest_idx = peak2_idx
            else:
                matches.append((peak1_idx, peak2_idx))
                # print((peak1_idx, peak2_idx))
    # print(matches)
    return matches


@numba.njit(fastmath=True)
def score_best_matches(matching_pairs: np.ndarray, spec1: np.ndarray,
                       spec2: np.ndarray, mz_power: float = 0.0,
                       intensity_power: float = 1.0) -> Tuple[float, int]:
    """Calculate cosine-like score by multiplying matches. Does require a sorted
    list of matching peaks (sorted by intensity product)."""
    score = float(0.0)
    used_matches = int(0)
    used1 = set()
    used2 = set()
    for i in range(matching_pairs.shape[0]):
        if not matching_pairs[i, 0] in used1 and not matching_pairs[i, 1] in used2:
            score += matching_pairs[i, 2]
            used1.add(matching_pairs[i, 0])  # Every peak can only be paired once
            used2.add(matching_pairs[i, 1])  # Every peak can only be paired once
            # print(i, matching_pairs[i,0], matching_pairs[i,1], used_matches, score)
            used_matches += 1

    # Normalize score:
    spec1_power = spec1[:, 0] ** mz_power * spec1[:, 1] ** intensity_power    
    spec2_power = spec2[:, 0] ** mz_power * spec2[:, 1] ** intensity_power

    # print(spec1_power)
    # print(spec2_power)
    # raise
    score_norm = (np.sum(spec1_power ** 2) ** 0.5 * np.sum(spec2_power ** 2) ** 0.5)
    # print(score, score_norm, used_matches)
    score = score/score_norm
    # print(score, "/", score_norm)
    # raise
    return score, used_matches

@numba.njit
def collect_peak_pairs(spec1: np.ndarray, spec2: np.ndarray,
                       tolerance: float, shift: float = 0, mz_power: float = 0.0,
                       intensity_power: float = 1.0):
    # pylint: disable=too-many-arguments
    """Find matching pairs between two spectra.

    Args
    ----
    spec1:
        Spectrum peaks and intensities as numpy array.
    spec2:
        Spectrum peaks and intensities as numpy array.
    tolerance
        Peaks will be considered a match when <= tolerance appart.
    shift
        Shift spectra peaks by shift. The default is 0.
    mz_power:
        The power to raise mz to in the cosine function. The default is 0, in which
        case the peak intensity products will not depend on the m/z ratios.
    intensity_power:
        The power to raise intensity to in the cosine function. The default is 1.

    Returns
    -------
    matching_pairs : numpy array
        Array of found matching peaks.
    """
    matches = find_matches(spec1[:, 0], spec2[:, 0], tolerance, shift)
    # global a
    # a = matches
    # matches_op = find_matches_opt(spec1[:, 0], spec2[:, 0], tolerance, shift)
    # global b
    # b = matches_op
    # assert np.allclose(matches, matches_op)
    
    idx1 = [x[0] for x in matches]
    idx2 = [x[1] for x in matches]
    if len(idx1) == 0:
        return None
    matching_pairs = []
    for i, idx in enumerate(idx1):
        power_prod_spec1 = (spec1[idx, 0] ** mz_power) * (spec1[idx, 1] ** intensity_power)
        power_prod_spec2 = (spec2[idx2[i], 0] ** mz_power) * (spec2[idx2[i], 1] ** intensity_power)
        # print((idx, idx2[i], power_prod_spec1 * power_prod_spec2))
        matching_pairs.append([idx, idx2[i], power_prod_spec1 * power_prod_spec2])
    return np.array(matching_pairs.copy())


start_collect_peaks = time.time()
pairs_to_score_list = []
scores = []

# We use this array to compare results from CPU to results from GPU
out_true = np.full((len(references), len(queries), 2), fill_value=-1, dtype='float32')

spec1s = []
for i,spectrum_1 in enumerate(references):
    spec1s.append([i, spectrum_1.peaks.to_numpy])
spec2s = []
for j,spectrum_2 in enumerate(queries):
    spec2s.append([j, spectrum_2.peaks.to_numpy])
total=len(spec1s) * len(spec2s)
for (i, spec1), (j, spec2) in tqdm( product(spec1s, spec2s), total=total):
        matching_pairs = collect_peak_pairs(
                    spec1, 
                    spec2, 
                    tolerance=0.1,
                    shift=0.0, 
                    mz_power=0.0,
                    intensity_power=1.0
        )
        if matching_pairs is not None:
            matching_pairs = matching_pairs[np.argsort(matching_pairs[:, 2])[::-1], :] 
            score = score_best_matches(matching_pairs, spec1, spec2, 0.0, 1.0)
            scores.append(score)
            out_true[i,j,0] = score[0]
            out_true[i,j,1] = score[1]

duration = time.time() - start_collect_peaks
print("Time to collect matching pairs:", duration)
persec = total / duration
print(f"Speed at {persec:.1f} pairs/sec")
print(f"Estimated {(100_000 * 1_500_000 / persec) / 3600:.2f}hrs per 100k x 1.5mln")


100%|██████████| 1048576/1048576 [00:13<00:00, 76640.76it/s]

Time to collect matching pairs: 13.7310631275177
Speed at 76365.2 pairs/sec
Estimated 545.62hrs per 100k x 1.5mln


# GPU-based solution

In [43]:
tolerance: float = 0.1
shift: float = 0
mz_power: float = 0
int_power: float = 1

rspec, references_cutoff = spectra_peaks_to_tensor(references, fill=-1e6)
qspec, queries_cutoff  = spectra_peaks_to_tensor(queries, fill=-1e6)

rspec_cu = cuda.to_device(rspec)
qspec_cu = cuda.to_device(qspec)

lens_cu = cuda.to_device(np.stack([references_cutoff, queries_cutoff]))

_,R,N = rspec_cu.shape
_,Q,M = qspec_cu.shape
rspec.shape

CudaAPIError: [999] Call to cuMemAlloc results in CUDA_ERROR_UNKNOWN

In [44]:
# Threads per block (max is 32,32)
# Blocks per grid (enough to cover all Rs and Qs)
TPB = (32, 32)
BPG_x = math.ceil(R / TPB[0])
BPG_y = math.ceil(Q / TPB[1])
BPG = (BPG_x, BPG_y)

# Smaller match limit increases speed dramatically!
# But any value less than M * N will cause a tiny portion of 
# pairs to "overflow" and have tiny error in calculation.
# It is much more efficient to calculate these overflows 
# separately rather than to increase MATCH_LIMIT.
MATCH_LIMIT = 128

@cuda.jit
def process(
            rspec: DeviceNDArray,
            qspec: DeviceNDArray,
            
            lens: DeviceNDArray,          
            
            out: DeviceNDArray,
            overflow: DeviceNDArray,
            ):
    
    i,j = cuda.grid(2)
    thread_i = cuda.threadIdx.x
    thread_j = cuda.threadIdx.y
    block_size_x = cuda.blockDim.x
    block_size_y = cuda.blockDim.y
    
    # mem = cuda.shared.array((8, ))
    # We aren't out of the RxQ grid
    if i < R and j < Q:
        # mem = cuda.shared.array((4, 4, 4, 32), types.float32)
        rmz = rspec[0]
        rint = rspec[1]
        qmz = qspec[0]
        qint = qspec[1]
        # In this i,j, We get length of r and q spectrums 
        # since they are batched, there might be extra filler elements
        rlen = lens[0]
        qlen = lens[1]
        
        rleni = rlen[i]
        qlenj = qlen[j]
        
        spec1_mz = rmz[i]
        spec1_int = rint[i]
        
        spec2_mz = qmz[j]
        spec2_int = qint[j]
        
        lowest_idx = types.int32(0)
        num_match = types.int32(0)
        
        matches = cuda.local.array((2, MATCH_LIMIT), types.int16)
        
        for peak1_idx in range(rleni):
            mz = spec1_mz[peak1_idx]
                
            low_bound = mz - tolerance
            high_bound = mz + tolerance
            
            for peak2_idx in range(lowest_idx, qlenj):
                mz2 = spec2_mz[peak2_idx] + shift
                if mz2 > high_bound:
                    break
                if mz2 < low_bound:
                    lowest_idx = peak2_idx
                else:
                    if num_match < MATCH_LIMIT:
                        matches[0, num_match] = peak1_idx
                        matches[1, num_match] = peak2_idx
                        num_match += 1
                    else:
                        overflow[i, j, 0] = 1 # This is the errorcode for overflow
                        break

        if num_match == 0: 
            return
        
        # SLOW, calculate norm ( This should be done in several threads )
        # score_norm = types.float32(0.0)
        score_norm = types.float32(1.0)
        score_norm_spec1 = types.float32(0.0)
        score_norm_spec2 = types.float32(0.0)
        
        for peak1_idx in range(rleni):
            score_norm_spec1 += ((spec1_mz[peak1_idx] ** mz_power) * (spec1_int[peak1_idx] ** int_power)) ** 2
        for peak2_idx in range(qlenj):
            score_norm_spec2 += ((spec2_mz[peak2_idx] ** mz_power) * (spec2_int[peak2_idx] ** int_power)) ** 2
        score_norm = math.sqrt(score_norm_spec1 * score_norm_spec2)
        
        # Quite slow - Bubble sort (This should also be done in several threads)
        # We need two cases, bubble sort up to 50 elems is fine
        score = types.float32(0.0)
        used_matches = types.int32(0)
        for _ in range(0, num_match):
            max_prod = -1
            max_peak1_idx = -1
            max_peak2_idx = -1
            
            for sj in range(0, num_match):
                if matches[0, sj] >= 0:
                    peak1_idx = matches[0, sj]
                    peak2_idx = matches[1, sj]
                    
                    power_prod_spec1 = (spec1_mz[peak1_idx] ** mz_power) * (spec1_int[peak1_idx] ** int_power)
                    power_prod_spec2 = (spec2_mz[peak2_idx] ** mz_power) * (spec2_int[peak2_idx] ** int_power)
                    prod = power_prod_spec1 * power_prod_spec2
                    if prod > max_prod:
                        max_prod = prod
                        max_peak1_idx = peak1_idx
                        max_peak2_idx = peak2_idx

            if max_prod > 0:
                for sj in range(0, num_match):
                    if matches[0, sj] == max_peak1_idx or matches[1, sj] == max_peak2_idx:
                        matches[0, sj] = -1 # "Remove" it
                        matches[1, sj] = -1 # "Remove" it
                score += max_prod
                used_matches += 1
                
            if max_prod < 0:
                break
            
        score = score / score_norm
            
        out[i,j,0] = score
        out[i,j,1] = used_matches

In [34]:
iters = 32
out = np.full((R, Q, 2), fill_value=-1, dtype='float32')
overflow = np.full((R, Q, 1), fill_value=0, dtype='uint8')
out_cu = cuda.to_device(out)
overflow_cu = cuda.to_device(overflow)

# Warm up
process[BPG, TPB](
                rspec_cu, qspec_cu,
                lens_cu,
                out_cu, overflow_cu,
                )

duration = time.time()

# Iterate kernel `iter` times and average performance
for _ in tqdm(range(iters), desc="Run x32, to get avg perf."):
    process[BPG, TPB](
                    rspec_cu, qspec_cu,
                    lens_cu,
                    out_cu, overflow_cu,
                    )
    
    cuda.synchronize()
persec = iters * (R * Q) / (time.time() - duration)
out_cu.copy_to_host(out)
overflow_cu.copy_to_host(overflow)
print(f"Speed at {persec:.1f}/sec")
print(f"Estimated {(100_000 * 1_500_000 / persec) / 3600:.2f}hrs per 100k x 1.5mln")

non_overflow = (1-overflow)
out_underflow = out * non_overflow

# out_true = np.load('data/grid_outp.npy')
out_true_underflow = out_true * non_overflow

print("Perfectly correct?:", np.allclose(out, out_true))
print("Except overflows, pefectly correct?:", np.allclose(out_underflow, out_true_underflow))
print(f"Total comparisons: ", R * Q * 2)
tc = np.isclose(out[...,:2], out_true[...,:2])
print(f"Total correct: {(tc).sum()} ({tc.sum() / (R * Q * 2 ) * 100 :.6f}%)")
print(f"Total wrong: {(1-tc).sum()} ({(1-tc).sum() / (R * Q * 2 ) * 100 :.6f}%)")

print("Overflows ====")
print(f"Overflows at MATCH_LIMIT={MATCH_LIMIT} : {overflow.sum()}, {overflow.mean() * 100:.5f}%")

print("Matches =====")
print(f"% correct : {np.isclose(out_underflow[...,1], out_true_underflow[...,1]).mean() * 100:.5f}%")
print(f"% under : {(out_underflow[...,1] < out_true_underflow[...,1]).mean() * 100:.5f}%")
print(f"% over : {(out_underflow[...,1] > out_true_underflow[...,1]).mean() * 100:.5f}%")

print("Scores =====")
print(f"% correct : {np.isclose(out_underflow[...,0], out_true_underflow[...,0]).mean() * 100:.5f}%")


CudaAPIError: [999] Call to cuMemAlloc results in CUDA_ERROR_UNKNOWN